In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Produce one dataset with the columns: "user_name", "item", "price" from this data:

In [2]:
# sales
sales = pd.DataFrame({
    'id': [101, 102, 103, 104, 105],
    'client_id': [1, 2, 1, 3, 3],
    'product': ['TechBook Pro 15', 'ViewMax 24HD', 'KeyPro MX', 'TechBook Air', 'ClickOne Mini'],
    'price': [1200, 999, 599, 1500, 99]
})

# clients
clients = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'name': ['Vladimir', 'Sergei', 'Michael', 'Ivan', 'Daria'],
    'city_id': [10, 11, 10, 11, 10]
})

# cities
cities = pd.DataFrame({
    'id': [10, 11],
    'name': ['Miami', 'Atlanta']
})

In [3]:
#rename columns:
sales_renamed = sales.rename(columns = {'id': 'product_id'})
clients_renamed = clients.rename(columns = {'id': 'client_id', 'name': 'client_name'})
cities_renamed = cities.rename(columns = {'id': 'city_id', 'name': 'city_name'})

In [4]:
tdf = sales_renamed.merge(clients_renamed, on = 'client_id', how = 'inner')
df = tdf.merge(cities_renamed, on = 'city_id', how = 'inner')
df

,product_id,client_id,product,price,client_name,city_id,city_name
0,101,1,TechBook Pro 15,1200,Vladimir,10,Miami
1,102,2,ViewMax 24HD,999,Sergei,11,Atlanta
2,103,1,KeyPro MX,599,Vladimir,10,Miami
3,104,3,TechBook Air,1500,Michael,10,Miami
4,105,3,ClickOne Mini,99,Michael,10,Miami


In [5]:
final_df = df[['client_name', 'product', 'price']]
final_df

,client_name,product,price
0,Vladimir,TechBook Pro 15,1200
1,Sergei,ViewMax 24HD,999
2,Vladimir,KeyPro MX,599
3,Michael,TechBook Air,1500
4,Michael,ClickOne Mini,99


In [6]:
df1 = sales_renamed.merge(clients_renamed, on = 'client_id', how = 'outer')
df1

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,product_id,client_id,product,price,client_name,city_id
0,101.0,1,TechBook Pro 15,1200.0,Vladimir,10
1,103.0,1,KeyPro MX,599.0,Vladimir,10
2,102.0,2,ViewMax 24HD,999.0,Sergei,11
3,104.0,3,TechBook Air,1500.0,Michael,10
4,105.0,3,ClickOne Mini,99.0,Michael,10
5,NaN,4,NaN,NaN,Ivan,11
6,NaN,5,NaN,NaN,Daria,10


In [7]:
#print the names of clients who didn't buy anything:
res = df1.loc[df1['price'].isna()]
clints_no_purchase = res.client_name.to_list()
print("Clients who didn't buy anything:", clints_no_purchase)

Clients who didn't buy anything: ['Ivan', 'Daria']


In [8]:
#print users + product + city:
df3 = sales_renamed.merge(clients_renamed, on = 'client_id', how = 'inner')
df3 = df3.merge(cities_renamed, on = 'city_id', how = 'inner')
df3[['client_name', 'product', 'city_name']]

,client_name,product,city_name
0,Vladimir,TechBook Pro 15,Miami
1,Sergei,ViewMax 24HD,Atlanta
2,Vladimir,KeyPro MX,Miami
3,Michael,TechBook Air,Miami
4,Michael,ClickOne Mini,Miami


In [9]:
#print answer to what city has more purchases:
city = df3.city_name.value_counts()
max_city_name = city.idxmax()
print(f"The city with max purhcases is {max_city_name}.")

The city with max purhcases is Miami.


In [10]:
#print answer to what client has more purchases:
clients = df3.client_name.value_counts()
max_purchase = clients.max()
top_clients = clients[clients == max_purchase].index.tolist()
print(f"The max number of purhcases is {max_purchase} and clients are {top_clients}.")

The max number of purhcases is 2 and clients are ['Vladimir', 'Michael'].


In [11]:
#join clints and cities using method join():
new_clients = clients_renamed.set_index('city_id')
new_cities = cities_renamed.set_index('city_id')
new_df = new_clients.join(new_cities).reset_index()
new_df

,city_id,client_id,client_name,city_name
0,10,1,Vladimir,Miami
1,11,2,Sergei,Atlanta
2,10,3,Michael,Miami
3,11,4,Ivan,Atlanta
4,10,5,Daria,Miami
